In [3]:
import pandas as pd
from tqdm import tqdm
import openai
tqdm.pandas()

In [4]:
data = pd.read_excel("topic_df.xlsx")

In [ ]:
OPENAI_API = "YOUR_OPENAI_API"
client = openai.OpenAI(api_key=OPENAI_API)

In [6]:
data['author_type']

0        微博官方认证
1        微博个人认证
2        微博个人认证
3           NaN
4        微博个人认证
          ...  
37509       NaN
37510       NaN
37511    微博个人认证
37512       NaN
37513       NaN
Name: author_type, Length: 37514, dtype: object

In [7]:
data[data['author_type'] == '微博官方认证']

,author_id,author,author_type,content,content_url,date,source,repost,comments,likes,chain_author,content_len,position,index_in_chain,previous_author,previous_content
0,1.191965e+09,三联生活周刊,微博官方认证,【#进口药退出公立医院# 】2024年12月30日，距离开标日18天后，第十批国家药品集中采...,https://weibo.com/1191965271/P9pjHs3Y8?refer_f...,2025-01-13 18:30:00,NaN,21850.0,4442.0,9699.0,三联生活周刊,245,start,0,NaN,NaN
9,1.663937e+09,财新网,微博官方认证,【集采再震荡：#几分钱一片的仿制药是喜是忧#？】2024年12月13日，第十批国家组织药品集...,https://weibo.com/1663937380/Paa2wlL7I?refer_f...,2025-01-18 17:27:00,微博网页版,2017.0,166.0,3105.0,财新网,1106,start,0,NaN,NaN
24,1.663937e+09,财新网,微博官方认证,【#一企业回应仿制药数据造假#：系药监上传数据错误，已更新】关于集采药的争议不断升温，社交平...,https://weibo.com/1663937380/Pb4pkm1x7?refer_f...,2025-01-24 16:57:00,微博网页版,1314.0,200.0,4266.0,财新网,384,start,0,NaN,NaN
27,1.926910e+09,第一财经日报,微博官方认证,#医保局回应政协委员关注集采药疗效#【#京沪政协委员关注集采药疗效#，国家医保局：欢迎医务人...,https://weibo.com/1926909715/PagLaznKZ?refer_f...,2025-01-19 10:33:00,NaN,1246.0,830.0,8712.0,第一财经日报,288,start,0,NaN,NaN
37,1.663937e+09,财新网,微博官方认证,【“麻药不睡、血压不降、泻药不泻” #上海政协委员忧心集采药物疗效不稳#】2025年上海两会...,https://weibo.com/1663937380/PaamW8t0m?refer_f...,2025-01-18 18:17:00,微博网页版,785.0,270.0,3008.0,财新网,680,start,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37474,6.153839e+09,跑合网,微博官方认证,发布了头条文章：《周报： 市场依旧低迷，中药集采倒逼原料和成药价格双降！》 O周报： 市场...,https://weibo.com/6153839096/P7PUSrAIO?refer_f...,2025-01-03 10:32:00,微博 weibo.com,0.0,0.0,0.0,跑合网,53,start,0,NaN,NaN
37475,1.830136e+09,江海晚报,微博官方认证,【大利好！南通已有人享受到】1日起，江苏率先开展的无创产前基因检测服务集采结果在南通落地，与...,https://weibo.com/1830136467/P7PU1o4nu?refer_f...,2025-01-03 10:30:00,微博网页版,0.0,0.0,0.0,江海晚报,128,start,0,NaN,NaN
37477,6.352579e+09,全球财说GF,微博官方认证,#上市公司晚间公告# 中恒集团：控股子公司产品拟中选全国中成药采购联盟集采。 ​,https://weibo.com/6352578800/P7CyDcpVL?refer_f...,2025-01-02 00:32:00,iPhone 13 Pro,0.0,0.0,1.0,全球财说GF,33,start,0,NaN,NaN
37479,6.011309e+09,健识局,微博官方认证,印度只要2分钱！3分一片的集采药，成本是多少？#集采##仿制药# L健识局的微博视频 ​,https://weibo.com/6011309186/P7JzZexxz?refer_f...,2025-01-02 18:24:00,微博视频号,0.0,0.0,0.0,健识局,31,start,0,NaN,NaN


In [8]:
import hanlp
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
tok.dict_force = {'原研药', '中成药', '小红书', '集采'}

def preprocess_text(text):
    if pd.isna(text):
        return None
    if len(text) == 0:
        return None
    tokens = tok(text)
    return ' '.join(tokens)

In [9]:
data = data[data.content_len  > 5]
data['content'] = data['content'].astype("str")
data['content_seg'] = data['content'].progress_apply(preprocess_text)

100%|██████████| 34597/34597 [13:45<00:00, 41.93it/s]  


In [10]:
data['content_seg']

0        【 # 进口 药 退出 公立 医院 # 】 2024年12月30日 ， 距离 开标日 18天...
1        财新 的 报道 —— 上海 20名 政协 委员 联名 提案 ： 集采 之下 的 国产 仿制药...
2        “ 血压 不 降 、 麻醉 不 睡 、 泻药 不 泻 ” ， 瑞金医院 副 院长 郑民华 等...
3        推 上 看到 的 : 在 日本人 写 的 ABO 小说 里 ， 大体上 ， 抑制剂 费用 总...
4        这 两天 ， 北京 和 上海政协 委员 纷纷 提案 ， 希望 国家 关注 集采 药 效果 不...
                               ...                        
37509    医药 由于 集采 ， 还 在 下行 通道 ， 要 极致 耐心 ， 等到 极为 便宜 ， 才是...
37510    医院 集采 了 中药 注射液 ， 中成药 先 说 中药 注射液 ， 本质 不是 中药 ， 是...
37511    行业 财经 要闻 1 . 我国 首部 能源 法 正式 施行 2025年 将 新增 风电 光 ...
37512    转 ： 医药 集采 之后 ， 商业 保险公司 的 “ 百万 医疗 ” 就 基本上 失去 存在...
37513    外公 吃 国产 抗生素 吃 了 一个月 不见 好转 拖 成 肺部 感染 ， 被 救护车 送进...
Name: content_seg, Length: 34597, dtype: object

In [11]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

#from bertopic.backend import OpenAIBackend
#embedding_model = OpenAIBackend(client, "text-embedding-3-small")

#from transformers.pipelines import pipeline
#embedding_model = pipeline("feature-extraction", model="BAAI/bge-m3")

#embeddings = embedding_model.encode(data['content_seg'].tolist(), show_progress_bar=True)

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [12]:
from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [13]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=200, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
# 读取停用词
with open('stop_words.txt', 'r', encoding='utf-8') as f:
    stop_words = set(line.strip() for line in f if line.strip())

vectorizer_model = CountVectorizer(
    stop_words=list(stop_words),analyzer='word', 
    token_pattern=u"(?u)\\b\\w+\\b",
    min_df=2, ngram_range=(1, 1))

In [15]:
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
我有一个主题，其中包含以下文档：
[DOCUMENTS]
该主题由以下关键词描述：[KEYWORDS]

根据上述信息，提取一个简短但高度描述性、六个单词以内的英文语句。确保格式如下：
topic: <主题>...
"""
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
}


In [16]:
embeddings = embedding_model.encode(data['content_seg'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/1082 [00:00<?, ?it/s]

In [17]:
from bertopic import BERTopic

topic_model = BERTopic(
    # Pipeline models
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    # Hyperparameters
    top_n_words=10,
    verbose=True,
)

# Train model
topics, probs = topic_model.fit_transform(data['content_seg'], embeddings)

# Show topics
topic_model.get_topic_info()

2025-03-06 22:08:44,188 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-06 22:09:47,572 - BERTopic - Dimensionality - Completed ✓
2025-03-06 22:09:47,574 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-06 22:09:52,578 - BERTopic - Cluster - Completed ✓
2025-03-06 22:09:52,588 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 15/15 [00:09<00:00,  1.64it/s]
2025-03-06 22:10:10,081 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,KeyBERT,OpenAI,MMR,Representative_Docs
0,-1,9143,-1_集采_药_说_药品,"[集采, 药, 说, 药品, 原研药, 医院, 医生, 医保, 仿制药, 质量]","[国家医保局, 医保局, 医药, 原研药, 药效, 药企, 国产, 药, 三联生活周刊, 药物]",[Pharmaceutical procurement quality risk inves...,"[药品, 医保, 仿制药, 数据, 医药, 医疗, 进口, 行业, 国家医保局, 临床]",[【 针对 某些 集采 药品 可能 存在 质量 风险 等 问题 国家医保局 牵头 调查 】 ...
1,0,14685,0_集采_药_药品_质量,"[集采, 药, 药品, 质量, 医生, 医保, 说, 原研药, 仿制药, 价格]","[国家医保局, 医保局, 国产, 药效, 原研药, 医药, 麻药, 中药, 药, 政协]",[Drug procurement quality assurance...],"[医保, 原研药, 仿制药, 医疗, 国家医保局, 临床, 医药, 进口, 药企, 产品]",[# 调查 集采 药品 可能 存在 的 质量 风险 问题 # 【 针对 某些 集采 药品 可...
2,1,1945,1_铺展_上班_级别_指标,"[铺展, 上班, 级别, 指标, 受, 影响, 干部, 医院, 药品, 集采]","[医院, 药品, 级别, 上班, 受, 指标, 集采, 真理, 朴素, 铺展]",[Hospital working level not affected.],"[上班, 级别, 指标, 影响, 干部, 医院, 药品, 集采, chsnwwii, 真理]",[我 在 医院 上班 回答 你 ： 不能 ， 没有 这个 指标 ， 但是 干部 级别 的 不...
3,2,1557,2_解决_质量_集采_垃圾,"[解决, 质量, 集采, 垃圾, l, 三, 点, 保证, 支持, 制度]","[质量, 集采, 垃圾, 解决, 制度, 三, 点, 价格, l, 支持]",[Quality issues in group procurement...],"[解决, 质量, 集采, 垃圾, l, 三, 保证, 支持, 制度, 价格]","[集采 有 质量 问题 如何 解决 ？, 集采 有 质量 问题 如何 解决 ？, 集采 有 ..."
4,3,1552,3_自费_免疫药_关心_靶向药,"[自费, 免疫药, 关心, 靶向药, 癌症, 买, 开, 患者, 原研药, 靶向]","[癌症, 免疫药, 靶向药, 药业, 免疫, 药, 自费, 原研药, 买, 患者]",[Self-pay options for cancer patients...],"[免疫药, 癌症, 患者, 原研药, inmo, 数据中心, 养老, 业务, 孕产妇, 陕西华达]",[癌症 患者 自费 能 开 一些 原研药 吗 ， 一些 免疫药 和 靶向药 能 自费 买 吗...
5,4,1551,4_抵达_边缘_这下_恐惧,"[抵达, 边缘, 这下, 恐惧, 医护, 办法, 人员, 真的, 器械, 差]","[差, 难用, 恐慌, 恐惧, 医院, 风险, 医护, 太, 无聊, 这下]",[Reaching the edge of fear...],"[边缘, 恐惧, 医护, 差, 职业, 难用, 恐慌, 手套, 太, 无聊]","[这下 真的 抵达 恐惧 边缘 了 器械 质量 这么 差 医护 人员 也 没 办法 啊, 这..."
6,5,1129,5_无聊_特么_洋抖_在意,"[无聊, 特么, 洋抖, 在意, 小红书, 事, 幼稚, 照样, 美国人, 讨论]","[无聊, 在意, 小红书, 讨论, 事, 特么, 幼稚, 照样, 洋抖, 美国人]","[Boredom, obsession, cultural discussions, chi...","[无聊, 特么, 洋抖, 在意, 小红书, 事, 幼稚, 照样, 美国人, 讨论]","[比起 小红书 洋抖 那些 无聊 的 事 ， 我 特么 更 在意 这些 问题 。。。, 比起..."
7,6,939,6_不配_活_干部_计较,"[不配, 活, 干部, 计较, 畜生, 阶层, 说白了, 优秀, 怕, 领导]","[不配, 不用, 说白了, 计较, 畜生, 怕, 活, 领导, 死, 干部]",[Unworthy officials shouldn't be alive.],"[不配, 活, 干部, 畜生, 阶层, 说白了, 怕, 领导, 死, 需求]","[不是 干部 不配 活 着, 不是 干部 不配 活 着, 不是 干部 不配 活 着]"
8,7,423,7_洋抖_在意_事_不错,"[洋抖, 在意, 事, 不错, 稳定, 药效, 仿造, 正, 测, 药材]","[好多, 在意, 小红书, 事, 关心, 无聊, 一点, 情况, 屁话, 管用]",[Care about authentic medicine and quality.],"[在意, 稳定, 药效, 测, 药材, 中成药, 季节, 产地, 雨水, 博士]","[比起 红书 洋抖 那些 事 ， 我 更 在意 这些, 比起 红书 洋抖 那些 事 ， 我 ..."
9,8,359,8_害怕_假_真_吓人,"[害怕, 假, 真, 吓人, 走势, 眼中, 敢, 一点, 捧, 加]","[害怕, 吓人, 假, 真, 没, 敢, 一点, 眼中, 捧, 加]",[Fearful of real or fake...],"[害怕, 假, 吓人, 走势, 眼中, 敢, 一点, 医药股, 创新药, 没]","[真 的 假 的 …… 我 害怕, 真 的 假 的 …… 我 害怕, 真 的 假 的 …… ..."


In [18]:
for i in topic_model.get_topic_info()['OpenAI'].apply(lambda x: x[0]).tolist():
    print(i)

Pharmaceutical procurement quality risk investigation...
Drug procurement quality assurance...
Hospital working level not affected.
Quality issues in group procurement...
Self-pay options for cancer patients...
Reaching the edge of fear...
Boredom, obsession, cultural discussions, childish.
Unworthy officials shouldn't be alive.
Care about authentic medicine and quality.
Fearful of real or fake...
Chemical teacher questions aspirin quality...
Blood test appointment tomorrow at hospital.
Opposition towards procurement system improvements...
American influencer dominates Japanese medical discourse.
Self-paying for cancer immunotherapy drugs.


In [19]:
chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
chatgpt_topic_labels[-1] = "Outlier Topic"
topic_model.set_topic_labels(chatgpt_topic_labels)

In [ ]:
topic_model.topic_aspects_["OpenAI"].values()

dict_values([[('Pharmaceutical industry reform and progress...', 1)], [('Drug procurement quality concerns addressed...', 1)], [('Hospital staff discussing drug procurement...', 1)], [('Cancer patients buying immunotherapy and targeted drugs...', 1)], [('Resolve procurement quality issues effectively', 1)], [('Reaching edge of fear, medical staff...', 1)], [('Concerned about trivial matters overboard...', 1)], [('Unworthy cadres not fit to live...', 1)], [("Scared of what's real or fake...", 1)], [('Tomorrow hospital blood test follow-up', 1)], [('Herbal medicine selection and quality control...', 1)], [('American influence in Japanese healthcare...', 1)], [('Availability of self-funded cancer medications...', 1)]])

In [ ]:
topic_labels = topic_model.generate_topic_labels(
    nr_words=3, topic_prefix=False, word_length=10, separator=", "
)

In [ ]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                                 topic_prefix=False,
                                                 word_length=10,
                                                 separator=", ")
topic_model.set_topic_labels(topic_labels)


TypeError: 'dict' object is not callable

In [2]:
topic_model.visualize_barchart(custom_labels=True, top_n_topics= 20)

NameError: name 'topic_model' is not defined

In [119]:
topic_model.visualize_hierarchy(custom_labels=True)

In [37]:
# Visualize hierarchy with custom labels
topic_model.visualize_hierarchy()

In [28]:
topics_over_time = topic_model.topics_over_time(data['content'], data['date'], nr_bins=20)

20it [00:00, 62.38it/s]


In [54]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)


In [ ]:
# Run the visualization with the original embeddings
topic_model.visualize_hierarchical_documents(data['content'], hierarchical_topics, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_hierarchical_documents(docs, hierarchical_topics, reduced_embeddings=reduced_embeddings)

In [58]:
topic_model.visualize_barchart()


In [39]:
topic_distr, _ = topic_model.approximate_distribution(data['content'], window=8, stride=4)


100%|██████████| 35/35 [00:00<00:00, 55.60it/s]


In [43]:
topic_model.visualize_hierarchy(custom_labels=True)